# 数据安全

### 成员推理攻击（Membership Inference Attack）
成员推理攻击的攻击目标是判断一个输入样本**是否存在于模型的训练数据集**中，可能会导致数据泄露、隐私泄露等风险。由于人工智能模型易过拟合训练数据、训练数据与测试数据分布的差异性等原因，使得深度学习模型易遭受成员推理攻击。

本节实验使用影子模型攻击来实现简单的成员推理攻击。其主要想法是，训练多个“影子模型”，通过影子模型来模拟受害者模型的表现，然后根据影子模型的训练情况构建攻击数据集。使用该攻击数据集训练的二分类模型可以判断一个输入样本是否是来自受害者模型的训练集。

其主要流程如下图所示：
<p align="center">
<img src="./src/overview.png" style="center" width="70%">
<center>影子模型攻击流程</center>
</p>

### 1. 训练受害者模型
首先使用 sklearn.datasets.load_wine() 加载葡萄酒分类数据集。

葡萄酒分类数据集是一个非常小且简单的数据集，其具体信息如下：
数据集包含 178 个样本，每个样本拥有 13 种不同的化学特征（即每个样本有 13 维特征），包括酸度、灰分，酒精浓度等。然后一共有三种不同类别的葡萄酒，对应标签0到2。

详细信息可参考[这个网页](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_wine.html#sklearn.datasets.load_wine)

In [1]:
# 加载葡萄酒分类数据集
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm
import numpy as np
import random

data, target = load_wine(return_X_y=True)
# 将每一维度缩放至0到1
scaler = MinMaxScaler()
data = scaler.fit_transform(data)
# 查看数据集的形状
print("数据集的形状:", data.shape)
# 查看数据集的前5行
print("数据集的前5行:\n", data[:5])
# 查看数据集的标签
print("数据集的标签:", target[:5])

数据集的形状: (178, 13)
数据集的前5行:
 [[0.84210526 0.1916996  0.57219251 0.25773196 0.61956522 0.62758621
  0.57383966 0.28301887 0.59305994 0.37201365 0.45528455 0.97069597
  0.56134094]
 [0.57105263 0.2055336  0.4171123  0.03092784 0.32608696 0.57586207
  0.51054852 0.24528302 0.27444795 0.26450512 0.46341463 0.78021978
  0.55064194]
 [0.56052632 0.3201581  0.70053476 0.41237113 0.33695652 0.62758621
  0.61181435 0.32075472 0.75709779 0.37542662 0.44715447 0.6959707
  0.64693295]
 [0.87894737 0.23913043 0.60962567 0.31958763 0.4673913  0.98965517
  0.66455696 0.20754717 0.55835962 0.55631399 0.30894309 0.7985348
  0.85734665]
 [0.58157895 0.36561265 0.80748663 0.53608247 0.52173913 0.62758621
  0.49578059 0.49056604 0.44479495 0.25938567 0.45528455 0.60805861
  0.32596291]]
数据集的标签: [0 0 0 0 0]


使用随机森林分类器来作为被攻击的目标模型（受害者模型）。
随机森林分类器的具体api参数内容可以参考[sklearn的官方网站](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)

该分类器使用 .fit(X, y) 函数进行训练，使用 .predict(X) 函数来预测对应标签，使用 .predict_proba(X) 函数来获取每个标签的预测概率值。其中 X 的形状为（批次，特征数）。

In [2]:
victim_model = RandomForestClassifier(n_estimators=100)
# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.4, random_state=42)
# 训练模型
victim_model.fit(X_train, y_train)
# 测试模型准确性
accuracy = victim_model.score(X_test, y_test)
print("模型准确性:", accuracy)

模型准确性: 1.0


定义查询模型的函数：输入样本 $x \rightarrow$ 预测概率向量 $prob$ ，来模拟通过api访问模型的应用场景

In [25]:
# 模拟通过api访问模型，得到模型的概率预测输出
def query_model(model, x):
    return model.predict_proba(x)

### 2. 合成数据集
由于攻击者无从得知受害者模型的训练集，而训练一个判断输入样本是否属于训练集的二分类模型需要相应的数据。因此，文章借助爬山算法来合成影子模型的训练数据集。具体算法如下：
<p align="center">
<img src="./src/synthesis.png" style="center" width="40%">
<center>合成数据集的算法</center>
</p>
值得注意的是，该算法运行时间较长，面对具有更多特征的数据集（如图像数据集）并不适用。文章提出在那些场景下，攻击者可能需要具有部分训练数据集的知识，如获得同分布但是无交集的数据集、或者可以得到扰动后的数据集。

In [4]:
# 合成数据集，适用于无法获取真实数据集的情况
# 对于 features 较多的情况不适用
def synthesize_data(model, label, num_iter, conf_min, rej_max, k_max, k_min, num_features):
    x = np.random.uniform(0, 1, num_features).reshape(1, -1)
    last_y_c = 0
    k = k_max
    j = 0
    for i in range(num_iter):
        pred = query_model(model, x)
        y_c = pred[:, label].item()
        if y_c > last_y_c:
            if y_c > conf_min and np.argmax(pred) == label:
                if random.random() < y_c:
                    return x.squeeze(0)
            last_y_c = y_c
            j = 0
        else:
            j += 1
            if j > rej_max:
                k = max(k_min, int(k / 2))
                j = 0
        # 随机变换 x 的 k 个像素
        idx_to_change = np.random.randint(0, x.shape[1], size=k)
        x[:, idx_to_change] = np.random.uniform(0, 1, size=(x.shape[0], k))
    return None

合成影子模型训练所需数据集

In [5]:
def get_synthetic_data(num_samples, num_features, model, num_classes, num_iter=1000, conf_min=0.8, rej_max=10, k_max=3, k_min=1):
    # 生成合成数据
    synthesize_data_list = []
    label_list = []
    for i in tqdm(range(num_samples // num_classes)):
        for label in range(num_classes):
            x = None
            while x is None:
                x = synthesize_data(model, label, num_iter
                                    , conf_min, rej_max, k_max, k_min, num_features)
            synthesize_data_list.append(x)
            label_list.append(label)
    return synthesize_data_list, label_list

num_classes = len(np.unique(y_train))

synthesize_data_train, synthesize_label_train = get_synthetic_data(num_samples=100, num_features=X_train.shape[1], model=victim_model, num_classes=num_classes)
# 将合成数据转换为 numpy 数组
synthesize_data_train, synthesize_label_train = np.array(synthesize_data_train), np.array(synthesize_label_train)

synthesize_data_test, synthesize_label_test = get_synthetic_data(num_samples=100, num_features=X_test.shape[1], model=victim_model, num_classes=num_classes)
synthesize_data_test, synthesize_label_test = np.array(synthesize_data_test), np.array(synthesize_label_test)

100%|██████████| 33/33 [06:02<00:00, 10.99s/it]


In [6]:
# 保存合成数据集
import os
if not os.path.exists('./data'):
    os.makedirs('./data')
np.save('./data/synthesize_data_train.npy', synthesize_data_train)
np.save("./data/synthesis_label_train.npy", synthesize_label_train)
np.save('./data/synthesize_data_test.npy', synthesize_data_test)
np.save("./data/synthesis_label_test.npy", synthesize_label_test)
print("合成数据集已保存")

合成数据集已保存


In [4]:
synthesize_data_train = np.load('./data/synthesize_data_train.npy')
synthesize_label_train = np.load("./data/synthesis_label_train.npy")
synthesize_data_test = np.load('./data/synthesize_data_test.npy')
synthesize_label_test = np.load("./data/synthesis_label_test.npy")
print("合成数据集已加载")

合成数据集已加载


In [5]:
print(synthesize_data_train.shape)
print(synthesize_label_train.shape)

(99, 13)
(99,)


### 3. 训练影子模型
使用合成数据集训练影子模型，并且构建训练攻击模型所用的二分类数据集

由于攻击者不知道受害者模型训练的具体信息，而为了训练攻击的二分类模型（用于判断一个给定的输入是否在训练集中），攻击者需要构建一个（预测向量，是否在训练数据集中）的数据集来训练攻击模型。

攻击者可以通过构建多个影子模型，并且将合成数据集分为多份（彼此可重叠）来训练不同的影子模型，使得构建的攻击数据集更多样化。

而对于影子模型，攻击者知道一个样本是否是来自其训练数据集，因此可以构建训练攻击模型所需的数据集。影子模型的数量越多，产生的样本对越多，越利于攻击的二分类模型的训练。

对于攻击数据集，其每一个样本包含三部分，模型的预测向量 $\mathbf{y}$，真实标签 $y$，以及其是否在训练数据集中（0代表不在训练集中，1代表在训练集中）：
$$
D_{attack}=\{(\mathbf{y} ,y) , \text{in-or-out} \}_{i=1}^n
$$
其中 in-or-out 是0-1变量，代表该样本对是否在训练集中。

In [18]:
# 将合成数据集分成 num_shared 份，可重叠
def split_dataset(dataset, true_label, num_shadows, num_classes, length):
    splits = []
    labels = []
    for i in range(num_shadows):
        split = []
        label = []
        for j in range(num_classes):
            idx = np.where(true_label == j)[0]
            if len(idx) < length:
                length = len(idx)
            selected_idx = np.random.choice(idx, size=length, replace=False)
            split.append(dataset[selected_idx])
            label.append(true_label[selected_idx])
        splits.append(np.concatenate(split, axis=0))
        labels.append(np.concatenate(label, axis=0))
    return splits, labels

In [19]:
num_classes = len(np.unique(synthesize_label_train))
num_shadows = 20
attack_X = []
attack_y = []

shadow_train_list, train_label_list = split_dataset(synthesize_data_train, synthesize_label_train, num_shadows, num_classes, len(synthesize_data_train) // 2)
shadow_test_list, test_label_list = split_dataset(synthesize_data_test, synthesize_label_test, num_shadows, num_classes, len(synthesize_data_test) // 2)
# 训练影子模型
for i in range(num_shadows):
    shadow_model = RandomForestClassifier(n_estimators=100, random_state=i)
    shadow_model.fit(shadow_train_list[i], train_label_list[i])
    # 测试影子模型的准确性
    shadow_model_accuracy = shadow_model.score(shadow_test_list[i], test_label_list[i])
    print(f"Shadow model {i} accuracy: {shadow_model_accuracy}")

    # 使用影子模型生成攻击二分类数据集
    in_pred = query_model(shadow_model, shadow_train_list[i])
    in_features = np.hstack((in_pred, train_label_list[i].reshape(-1, 1)))
    attack_X.append(in_features)
    attack_y.append(np.ones((in_pred.shape[0], 1)))

    out_pred = query_model(shadow_model, shadow_test_list[i])
    out_features = np.hstack((out_pred, test_label_list[i].reshape(-1, 1)))
    attack_X.append(out_features)
    attack_y.append(np.zeros((out_pred.shape[0], 1)))
print("successfully generate attack dataset")

Shadow model 0 accuracy: 1.0
Shadow model 1 accuracy: 1.0
Shadow model 2 accuracy: 1.0
Shadow model 3 accuracy: 1.0
Shadow model 4 accuracy: 1.0
Shadow model 5 accuracy: 1.0
Shadow model 6 accuracy: 1.0
Shadow model 7 accuracy: 1.0
Shadow model 8 accuracy: 1.0
Shadow model 9 accuracy: 1.0
Shadow model 10 accuracy: 1.0
Shadow model 11 accuracy: 1.0
Shadow model 12 accuracy: 1.0
Shadow model 13 accuracy: 1.0
Shadow model 14 accuracy: 1.0
Shadow model 15 accuracy: 1.0
Shadow model 16 accuracy: 1.0
Shadow model 17 accuracy: 1.0
Shadow model 18 accuracy: 1.0
Shadow model 19 accuracy: 1.0
successfully generate attack dataset


使用合成数据集构建二分类所用数据集

In [20]:
attack_X = np.vstack(attack_X)
attack_y = np.vstack(attack_y)
print("attack_X shape:", attack_X.shape)
print("attack_y shape:", attack_y.shape)

attack_X shape: (3960, 4)
attack_y shape: (3960, 1)


### 4. 训练攻击二分类模型
接下来，借助影子模型构建的攻击数据集，我们可以训练二分类模型。提出影子模型攻击的论文指出，该二分类模型是独立于之前的影子模型训练过程的，因此可以根据任务需求选择适合的模型进行训练。

这里我们依然采用随机森林分类器作为二分类的攻击模型。

In [21]:
# 训练攻击模型
attack_model = RandomForestClassifier(n_estimators=100, random_state=42)
attack_model.fit(attack_X, attack_y)

d:\python\lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestClassifier(random_state=42)

接下来在真实的训练集以及测试集（即受害者模型所训练的数据集以及测试集）上测试我们的攻击效果。

In [27]:
# 测试攻击模型
def test_attack_model(attack_model, victim_model, X_train, y_train, X_test, y_test):
    num_train = 0
    num_test = 0
    acc_train = 0
    acc_test = 0
    
    preds = query_model(victim_model, X_train)
    features = np.hstack((preds, y_train.reshape(-1, 1)))
    train_pred = attack_model.predict(features)
    num_train += len(train_pred)
    acc_train += np.sum(train_pred == 1)

    preds = query_model(victim_model, X_test)
    features = np.hstack((preds, y_test.reshape(-1, 1)))
    test_pred = attack_model.predict(features)
    num_test += len(test_pred)
    acc_test += np.sum(test_pred == 0)
    print("训练集攻击模型准确率:", acc_train / num_train)
    print("测试集攻击模型准确率:", acc_test / num_test)
    print("\n一些指标：")
    accuracy = (acc_train + acc_test) / (num_train + num_test)
    print("攻击模型准确率 accuracy:", accuracy)
    precision = acc_train / (acc_train + num_test - acc_test)
    print("精确率 precision:", precision)
    recall = acc_train / num_train
    print("召回率 recall:", recall)
    print("F1-score:", 2 * precision * recall / (precision + recall))
    
test_attack_model(attack_model, victim_model, X_train, y_train, X_test, y_test)

训练集攻击模型准确率: 0.6981132075471698
测试集攻击模型准确率: 0.625

一些指标：
攻击模型准确率 accuracy: 0.6685393258426966
精确率 precision: 0.7326732673267327
召回率 recall: 0.6981132075471698
F1-score: 0.714975845410628


一个随机猜测的二分类判断器准确率为 $50%$，而我们的攻击分类器达到了 $67\%$，说明该攻击有一定效果。

此外，影子模型攻击存在诸多限制：
- 攻击者需要知道目标模型结构
- 目标模型需要过拟合才能达到较好的攻击效果
- 局限于特征数少的分类任务

### 5. 简单的防御思路
同上一次实验种，模型窃取的防御思路一致。可以采取模糊化模型输出向量的方式进行防御，使模型输出包含尽可能少的信息。可以将 query_model() 函数的输出换成模糊后的概率向量来模拟模糊化输出的过程。

In [23]:
# 模拟通过api访问模型，得到模糊化后的概率预测输出
def query_model(model, x):
    result = model.predict_proba(x)
    # 模糊化处理，保留 1 位小数
    # 最大的预测向量不变，其余均设置为 0
    max_idx = np.argmax(result, axis=1)
    result = np.round(result, 1)
    # 将最大值以外的元素设置为 0
    for i in range(result.shape[0]):
        for j in range(result.shape[1]):
            if j != max_idx[i]:
                result[i, j] = 0.0
    return result